In [1]:
import fastf1
from fastf1 import get_session
from fastf1.core import InvalidSessionError
from datetime import datetime

In [2]:
import pandas as pd
import numpy as np

In [3]:
def get_race_data(year, num_races):
    races = []
    for race in range(1, num_races):  # Maximum of 22 races in a season
        try:
            session = fastf1.get_session(year, race, 'R')
            session.load(telemetry=False)
            print(session.session_info)
            weather_data = session.laps.get_weather_data()
            #print(weather_data)
            laps = session.laps
            laps = laps.reset_index(drop=True)
            weather_data = weather_data.reset_index(drop=True)
    
            # exclude the 'Time' column from weather data when joining
            race_data = pd.concat([laps, weather_data.loc[:, ~(weather_data.columns == 'Time')]], axis=1)
            race_data['Year'] = year
            race_data['Event'] = session.session_info['Meeting']['Name']
            races.append(race_data)
        except InvalidSessionError:
            # No such race found for the given year
            # Could be because there was no race held in that place in that year
            break
    return races

In [4]:
#https://stackoverflow.com/questions/21414639/convert-timedelta-to-floating-point
def convert_timedelta_to_seconds(df):
    for column in df.select_dtypes(include=['timedelta64']):
        df[column] = df[column].dt.total_seconds()
    return df

In [5]:
def clean_data(races_data):
    races_data = races_data.loc[races_data['IsAccurate'] == True]
    races_data = races_data.loc[races_data['FastF1Generated'] == False]
    races_data = races_data.loc[races_data['Deleted'] == False]
    #https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html
    races_data = races_data.drop(columns=['LapStartDate','PitInTime', 'PitOutTime','DeletedReason','IsAccurate','FastF1Generated',
                                          'Deleted','Sector1SessionTime','Sector2SessionTime','Sector3SessionTime','LapStartTime'])
    #https://stackoverflow.com/questions/15891038/change-column-type-in-pandas
    races_data[['AirTemp', 'Humidity','Pressure','TrackTemp','WindDirection','WindSpeed']] = races_data[['AirTemp', 'Humidity','Pressure','TrackTemp','WindDirection','WindSpeed']].apply(pd.to_numeric)
    races_data = races_data.astype({'Rainfall': bool})
    
    races_data.dropna(inplace = True)
    
    # Convert timedelta64 columns to seconds
    races_data = convert_timedelta_to_seconds(races_data)
    return races_data

In [6]:
# Define the range of years
years = [2020, 2021, 2022, 2023, 2024]
num_races_in_year = [17, 22, 22, 22, 24]
years_dict = {years[i]: num_races_in_year[i] for i in range(len(years))}
races = []
for year, num_races in years_dict.items():
    year_data = pd.concat(get_race_data(year, num_races))
    year_data_cleaned = clean_data(year_data)
    races.append(year_data_cleaned)
races_data = pd.concat(races)

req         WARNING 	DEFAULT CACHE ENABLED! (599.4 MB) C:\Users\abhir\AppData\Local\Temp\fastf1
core           INFO 	Loading data for Austrian Grand Prix - Race [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Driver 77: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 16: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver  4: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 44: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 55: Lap timing integrity ch

{'Meeting': {'Key': 1045, 'Name': 'Austrian Grand Prix', 'Location': 'Spielberg', 'Country': {'Key': 17, 'Code': 'AUT', 'Name': 'Austria'}, 'Circuit': {'Key': 19, 'ShortName': 'Spielberg'}}, 'ArchiveStatus': {'Status': 'Generating'}, 'Key': 5744, 'Type': 'Race', 'Name': 'Race', 'StartDate': datetime.datetime(2020, 7, 5, 15, 10), 'EndDate': datetime.datetime(2020, 7, 5, 17, 10), 'GmtOffset': datetime.timedelta(seconds=7200), 'Path': '2020/2020-07-05_Austrian_Grand_Prix/2020-07-05_Race/'}


core           INFO 	Loading data for Styrian Grand Prix - Race [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['44', '77', '33', '23', '4', '11', '18', '3', '55', '26', '7', '20', '8', '99', '10', '63', '6', '31', '16', '5']


{'Meeting': {'Key': 1046, 'Name': 'Styrian Grand Prix', 'Location': 'Spielberg', 'Country': {'Key': 17, 'Code': 'AUT', 'Name': 'Austria'}, 'Circuit': {'Key': 19, 'ShortName': 'Spielberg'}}, 'ArchiveStatus': {'Status': 'Generating'}, 'Key': 5752, 'Type': 'Race', 'Name': 'Race', 'StartDate': datetime.datetime(2020, 7, 12, 15, 10), 'EndDate': datetime.datetime(2020, 7, 12, 17, 10), 'GmtOffset': datetime.timedelta(seconds=7200), 'Path': '2020/2020-07-12_Styrian_Grand_Prix/2020-07-12_Race/'}


core           INFO 	Loading data for Hungarian Grand Prix - Race [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['44', '33', '77', '18', '23', '5', '11', '3', '55', '20', '16', '26', '4', '31', '7', '8', '99', '63', '6', '10']


...

{'Meeting': {'Key': 1249, 'Name': 'São Paulo Grand Prix', 'OfficialName': 'FORMULA 1 LENOVO GRANDE PRÊMIO DE SÃO PAULO 2024', 'Location': 'São Paulo', 'Number': 21, 'Country': {'Key': 10, 'Code': 'BRA', 'Name': 'Brazil'}, 'Circuit': {'Key': 14, 'ShortName': 'Interlagos'}}, 'ArchiveStatus': {'Status': 'Generating'}, 'Key': 9636, 'Type': 'Race', 'Name': 'Race', 'StartDate': datetime.datetime(2024, 11, 3, 12, 30), 'EndDate': datetime.datetime(2024, 11, 3, 14, 30), 'GmtOffset': datetime.timedelta(days=-1, seconds=75600), 'Path': '2024/2024-11-03_São_Paulo_Grand_Prix/2024-11-03_Race/'}


core           INFO 	Loading data for Las Vegas Grand Prix - Race [v3.4.4]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data

{'Meeting': {'Key': 1250, 'Name': 'Las Vegas Grand Prix', 'OfficialName': 'FORMULA 1 HEINEKEN SILVER LAS VEGAS GRAND PRIX 2024', 'Location': 'Las Vegas', 'Number': 22, 'Country': {'Key': 19, 'Code': 'USA', 'Name': 'United States'}, 'Circuit': {'Key': 152, 'ShortName': 'Las Vegas'}}, 'ArchiveStatus': {'Status': 'Generating'}, 'Key': 9644, 'Type': 'Race', 'Name': 'Race', 'StartDate': datetime.datetime(2024, 11, 23, 22, 0), 'EndDate': datetime.datetime(2024, 11, 24, 0, 0), 'GmtOffset': datetime.timedelta(days=-1, seconds=57600), 'Path': '2024/2024-11-23_Las_Vegas_Grand_Prix/2024-11-23_Race/'}


core           INFO 	Loading data for Qatar Grand Prix - Race [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
Request for URL https://ergast.com/api/f1/2024/23/results.json failed; using cached response
Traceback (most recent call last):
  File "C:\Users\abhir\py312-venv\Lib\site-packages\requests_cache\session.py", line 286, in _resend
    response = self._send_and_cache(request, actions, cached_response, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\abhir\py312-venv\Lib\site-packages\requests_cache\session.py", line 254, in _send_and_cache
    response = super().send(request, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\abhir\py312-venv\Lib\site-packages\fastf1\req.py", line 134, in send
    lim.limit()
  File "C:\Users\abhir\py312-venv\Lib\site-packages\fastf1\req.py", line 104, in limit
    raise RateLimitExceededError(se

{'Meeting': {'Key': 1251, 'Name': 'Qatar Grand Prix', 'OfficialName': 'FORMULA 1 QATAR AIRWAYS QATAR GRAND PRIX 2024', 'Location': 'Lusail', 'Number': 23, 'Country': {'Key': 149, 'Code': 'QAT', 'Name': 'Qatar'}, 'Circuit': {'Key': 150, 'ShortName': 'Lusail'}}, 'ArchiveStatus': {'Status': 'Generating'}, 'Key': 9655, 'Type': 'Race', 'Name': 'Race', 'StartDate': datetime.datetime(2024, 12, 1, 19, 0), 'EndDate': datetime.datetime(2024, 12, 1, 21, 0), 'GmtOffset': datetime.timedelta(seconds=10800), 'Path': '2024/2024-12-01_Qatar_Grand_Prix/2024-12-01_Race/'}


In [7]:
races_data_copy = races_data

In [12]:
races_data.head(-1)

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,Sector1Time,Sector2Time,Sector3Time,SpeedI1,...,Position,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed,Year,Event
1,2118.765,GAS,10,72.412,2.0,1.0,17.960,32.228,22.224,282.0,...,12.0,27.7,35.5,939.0,False,54.1,276,0.7,2020,Austrian Grand Prix
2,2190.076,GAS,10,71.311,3.0,1.0,17.513,31.717,22.081,307.0,...,12.0,28.1,35.6,939.0,False,54.0,156,1.7,2020,Austrian Grand Prix
3,2260.801,GAS,10,70.725,4.0,1.0,17.410,31.510,21.805,309.0,...,12.0,28.6,33.7,939.0,False,54.1,90,2.0,2020,Austrian Grand Prix
4,2332.312,GAS,10,71.511,5.0,1.0,17.464,31.827,22.220,303.0,...,12.0,28.9,32.8,938.9,False,54.7,87,3.1,2020,Austrian Grand Prix
5,2404.340,GAS,10,72.028,6.0,1.0,17.944,31.741,22.343,273.0,...,12.0,28.3,34.0,938.9,False,53.6,72,2.9,2020,Austrian Grand Prix
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918,6320.318,PIA,81,85.284,33.0,1.0,31.705,28.931,24.648,238.0,...,3.0,18.6,57.0,1015.7,False,22.3,11,1.4,2024,Qatar Grand Prix
928,7592.320,PIA,81,86.116,43.0,4.0,32.589,28.879,24.648,238.0,...,4.0,18.4,61.0,1015.6,False,22.0,334,1.1,2024,Qatar Grand Prix
929,7676.621,PIA,81,84.301,44.0,4.0,31.523,28.442,24.336,240.0,...,4.0,18.3,62.0,1015.6,False,22.0,14,1.7,2024,Qatar Grand Prix
930,7760.601,PIA,81,83.980,45.0,4.0,31.390,28.358,24.232,239.0,...,3.0,18.3,62.0,1015.6,False,22.0,31,1.4,2024,Qatar Grand Prix


In [10]:
races_data.to_csv('f1races.csv')

In [11]:
races_data.columns

Index(['Time', 'Driver', 'DriverNumber', 'LapTime', 'LapNumber', 'Stint',
       'Sector1Time', 'Sector2Time', 'Sector3Time', 'SpeedI1', 'SpeedI2',
       'SpeedFL', 'SpeedST', 'IsPersonalBest', 'Compound', 'TyreLife',
       'FreshTyre', 'Team', 'TrackStatus', 'Position', 'AirTemp', 'Humidity',
       'Pressure', 'Rainfall', 'TrackTemp', 'WindDirection', 'WindSpeed',
       'Year', 'Event'],
      dtype='object')